In [35]:
import numpy as np
import pandas as pd
import re

In [46]:
data=pd.read_excel('../Data/한국마사회_경주상세정보(제주_부경)_20230720.xlsx')

In [47]:
unicol_list = []
for col in data.columns:
    if data[1:][col].nunique()<=1:
        unicol_list.append(col)
drop_data = data.drop(unicol_list,axis=1) 

In [76]:
unicol_list[24:]

['BU_2F-G_TIME',
 'BU_3F-G_TIME',
 'BU_1F-G_TIME',
 'BU_S1F_ACC_TIME',
 'BU_G8F_ACC_TIME',
 'BU_G6F_ACC_TIME',
 'BU_G4F_ACC_TIME',
 'BU_G3F_ACC_TIME',
 'BU_G2F_ACC_TIME',
 'BU_G1F_ACC_TIME']

In [48]:
drop_data["WG"] = drop_data["WG_HR"].str.extract(r'(\d+)(?=\()')
drop_data["WG"][0]="마체중"

drop_data["WC"] = drop_data["WG_HR"].str.extract(r'\((.*?)\)')
drop_data["WC"][0] = "체중증감"

drop_data["WC"][1:] = pd.to_numeric(drop_data["WC"][1:])
drop_data["WG"][1:] = pd.to_numeric(drop_data["WG"][1:])

drop_data = drop_data.drop(["WG_HR"],axis=1)

In [49]:
#한글 이름을 열로 바꿔줌
drop_data.columns=drop_data.iloc[0]
drop_data=drop_data[1:]
drop_data

,경주일자,경주번호,경주거리,경주등급,날씨,주로상태(함수율),착순,출주번호,마명,마번,...,"서울,제주G1F구간통과순위(서울,제주공통)",제주S-1F통과기록,제주1코너통과기록,제주2코너통과기록,제주3코너통과기록,제주G-3F통과기록,제주4코너통과기록,제주G-1F통과기록,마체중,체중증감
1,20230701,1,800,제6등급,흐림,포화 (18%),1,9,한백제왕,3101848,...,1,16.8,0,0,16.8,48.4,31.2,18.4,311,-18.0
2,20230701,1,800,제6등급,흐림,포화 (18%),2,2,명의로운,3102087,...,2,16.6,0,0,16.6,50.7,31.2,19.5,283,-6.0
3,20230701,1,800,제6등급,흐림,포화 (18%),3,3,번개의꿈,3101489,...,4,17.8,0,0,17.8,49.7,33,17.6,271,-5.0
4,20230701,1,800,제6등급,흐림,포화 (18%),4,6,고차원,3103528,...,5,17.6,0,0,17.6,50,33.2,17.7,265,-8.0
5,20230701,1,800,제6등급,흐림,포화 (18%),5,1,위너고,3102911,...,7,18.2,0,0,18.2,49.5,33.5,17.5,265,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29588,20190111,9,900,한4등급,흐림,양호 (8%),6,8,백년친구,1017787,...,3,14.7,0,0,21.4,42.7,34.9,15,308,5.0
29589,20190111,9,900,한4등급,흐림,양호 (8%),7,3,미래제주,1017443,...,4,15.2,0,0,21.9,42.3,35.2,14.9,311,-2.0
29590,20190111,9,900,한4등급,흐림,양호 (8%),8,1,남단여제,1017686,...,10,15.6,0,0,22.3,42.2,35.6,14.8,282,-4.0
29591,20190111,9,900,한4등급,흐림,양호 (8%),9,4,섬문화,1017765,...,9,15.7,0,0,22.5,42.1,35.9,14.9,316,4.0


In [50]:
#1착, 2착 횟수가 높다고 좋은 것이 아니라 승률이 높아야 좋다고 생각하기에, 경주마, 기수, 조교수, 위탁 각각의 승률을 새로운 열에 추가해주었다.
drop_data['경주마통산1착승률']=drop_data['경주마통산1착횟수']/drop_data['경주마통산출전횟수'].replace(0,1)
drop_data['경주마통산2찬승률']=drop_data['경주마통산2착횟수']/drop_data['경주마통산출전횟수'].replace(0,1)
drop_data['경주마최근1년1착승률']=drop_data['경주마최근1년1착횟수']/drop_data['경주마최근1년출전횟수'].replace(0,1)
drop_data['경주마최근1년2착승률']=drop_data['경주마최근1년2착횟수']/drop_data['경주마최근1년출전횟수'].replace(0,1)

drop_data['기수통산1착승률']=drop_data['기수통산1착횟수']/drop_data['기수통산출전횟수'].replace(0,1)
drop_data['기수통산2착승률']=drop_data['기수통산2착횟수']/drop_data['기수통산출전횟수'].replace(0,1)
drop_data['기수최근1년간1착승률']=drop_data['기수최근1년간1착횟수']/drop_data['기수최근1년간출전횟수'].replace(0,1)
drop_data['기수최근1년간2착승률']=drop_data['기수최근1년간2착횟수']/drop_data['기수최근1년간출전횟수'].replace(0,1)

drop_data['조교사통산1착승률']=drop_data['조교사통산1착횟수']/drop_data['조교사통산출전횟수'].replace(0,1)
drop_data['조교사통산2착승률']=drop_data['조교사통산2착횟수']/drop_data['조교사통산출전횟수'].replace(0,1)
drop_data['조교사최근1년간1착승률']=drop_data['조교사최근1년간1착횟수']/drop_data['조교사최근1년간출전횟수'].replace(0,1)
drop_data['조교사최근1년간2착승률']=drop_data['조교사최근1년간2착횟수']/drop_data['조교사최근1년간출전횟수'].replace(0,1)

drop_data['조교사위탁경주마1위승률']=drop_data['조교사위탁경주마1위횟수']/drop_data['조교사위탁경주마출전횟수'].replace(0,1)
drop_data['조교사위탁경주마2위승률']=drop_data['조교사위탁경주마2위횟수']/drop_data['조교사위탁경주마출전횟수'].replace(0,1)


In [51]:
#주로상태(함수율) 데이터가 포화(18%)로 되어 있는 것을 포화, 18로 바꿔주었다.
drop_data[['주로상태','주로상태_함수율']]=drop_data['주로상태(함수율)'].str.extract(r'(.*?) \((\d+)%\)')
drop_data=drop_data.drop('주로상태(함수율)',axis=1)

In [58]:
drop_data[['혈통','등급']]=drop_data['경주등급'].str.extract(r'(\D)(.*)')

In [65]:
drop_data['혈통']=drop_data['혈통'].str.replace('제','제주').str.replace('한','한라')

In [67]:
drop_data=drop_data.drop('경주등급',axis=1)

In [73]:
drop_data.to_excel('경마데이터.xls',index=False)

C:\Users\wjswp\AppData\Local\Temp\ipykernel_16784\3188981914.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  drop_data.to_excel('경마데이터.xls',index=False)
